In [1]:
"""给定一个字符串 S 和一个字符串 T，请在 S 中找出包含 T 所有字母的最小子串。
如果 S 中不存这样的子串，则返回空字符串 ""。
如果 S 中存在这样的子串，我们保证它是唯一的答案。
"""

'给定一个字符串 S 和一个字符串 T，请在 S 中找出包含 T 所有字母的最小子串。\n如果 S 中不存这样的子串，则返回空字符串 ""。\n如果 S 中存在这样的子串，我们保证它是唯一的答案。\n'

In [3]:
"""
【思路】和438很像
【错误】
1. isSubstring写错
2. 数组越界问题，此时的写法是维护一个滑动窗口[i,j)，恰当[i, len)也包含t，
此时j = len(s)。然后缩小窗口后，总会进入不符合的情况，此时要先判断s是否小于len(s)
否则直接调s[j]会数据越界。（最后一个元素也要判断，判断后注意数组越界）
【体会】
1. 注释部分为维护滑动窗口[i,j]，含义的变化，也是循环变量位置不一样
2. dict的get方法
"""


class Solution:
    def minWindow(self, s: str, t: str):
        d_t = self.counter(t)
        i, j = 0, 0
        d_w = self.counter("")
        min_len = len(s) + 1
        ret = ""
        # [i,j)
        while j <= len(s):
            # print(s[i: j], s[j])
            if self.isSubstring(d_t, d_w):
                # 滑动窗口已经包含t，尽量去缩小窗口
                if j - i <= min_len:
                    ret = s[i: j]
                    min_len = j - i

                d_w[s[i]] = d_w.get(s[i]) - 1
                if d_w[s[i]] == 0:
                    del d_w[s[i]]
                i += 1
                continue
            # 滑动窗口还未包含t，扩大窗口
            if j < len(s):
                d_w[s[j]] = d_w.get(s[j], 0) + 1
            j += 1
        return ret

    def counter(self, s: str):
        from collections import Counter
        return dict(Counter(s).items())

    def isSubstring(self, a, b):
        ret = True
        for i in a:
            if a.get(i) > b.get(i, 0):
                ret = False
                break
        return ret

# i, j = 0, -1
# # [i, j]
# while j < len(s):
#     if self.isSubstring(d_t, d_w):
#         # 滑动窗口已经包含t，尽量去缩小窗口
#         if j - i <= min_len:
#                     ret = s[i: j]
#                     min_len = j - i
#         d_w[s[i]] = d_w.get(s[i]) - 1
#         if d_w[s[i]] == 0:
#             del d_w[s[i]]
#         i += 1
#     else:
#         j += 1
#         if j<len(s):
#             d_w[s[j]] = d_w.get(s[j], 0) + 1


In [4]:
"""由于只有字母，使用数组而不是hashmap"""


class Solution:
    def minWindow(self, s: str, t: str):
        t_ = self.getMap(t)
        win = [0] * 58
        i, j = 0, 0
        min_len = len(s) + 1
        ret = ""
        #[i,j)
        while j <= len(s):
            if self.isSubstring(t_, win):
                if j - i < min_len:
                    min_len = j - i
                    ret = s[i:j]

                k = self.convert(s[i])
                win[k] -= 1
                i += 1
                continue
            if j < len(s):
                k = self.convert(s[j])
                win[k] += 1
            j += 1
        return ret

    def convert(self, s):
        return ord(s) - ord('A')

    def getMap(self, s):
        t = [0] * 58
        for i in s:
            t[self.convert(i)] += 1
        return t

    def isSubstring(self, a, b):
        for i in range(len(a)):
            if a[i] > b[i]:
                return False
        return True


In [ ]:
"""
【思路】虽然在s上维护了一个滑动窗口，但是每次窗口变动都要判断新的窗口是否包含t所有的字符。这次判断能不能再O(1)的时间完成？
也就是说，只需对新加的元素或者新删的元素判断，窗口内的其他元素不需要判断。
1. 之前做法是，s上动态维护一个滑动窗口w，用w不断和t比较，查看w是否包含t。
2. 现在想要只判断滑动窗口新加入或新删除的元素，查看w是否包含t。

【方法】来自leetcode讨论zza
0. 使用left， right 代表窗口[left, right)
1. mem字典中存储所有字母的元素及其频数，默认为0
2. 当t所有字符及其频数更新mem
3. 没遇到一个元素，mem对应字符的频数就减1
4. 当t所对应的字符的频数都为0，代表窗口内肯定包含t
5. 此时，想要右移left，缩小窗口，最理想的情况，直接缩到包含t的字符（如果t中某字符有k个，那么希望窗口只剩下k个改字母）
6. 可以发现，窗口内的非t中字符（或者t中某字符有k个，但窗口中该字符多余k个）的频数为负数--》（默认为0，遇到就减1，所以为负数）（初始为k，遇到m次，m>k,所以为k-m<0）
7. 直接左移到使mem[s[left]]=0,此时[left, right]就是一个合法窗口
8. *注意*，步骤4判断t中所对应的字符频数都为0，这个操作仍然不是O(1)的，使用t_len来记录遇到的t中字符的数目（初始为t的长度），只要t_len==0，就保证窗口合法。注意t_len的维护
"""

from collections import defaultdict


class Solution(object):
    def minWindow(self, s, t):
        """
        :type s: str
        :type t: str
        :rtype: str
        """
        mem = defaultdict(int)
        for char in t:
            mem[char] += 1
        t_len = len(t)

        minLeft, minRight = 0, len(s)
        left = 0

        for right, char in enumerate(s):
            if mem[char] > 0:  # 扩张滑动窗口
                t_len -= 1
            mem[char] -= 1

            if t_len == 0:
                # 滑动窗口已经包含t，去缩小
                while mem[s[left]] < 0:
                    mem[s[left]] += 1
                    left += 1

                if right - left < minRight - minLeft:
                    minLeft, minRight = left, right

                mem[s[left]] += 1
                t_len += 1
                left += 1
        return '' if minRight == len(s) else s[minLeft:minRight + 1]
